In [ ]:
# Add Path to Bitcoin Core source directory.
import util

# Bitcoin Core Test Framework 
from test_framework.key import SECP256K1_ORDER, SECP256K1_FIELD_SIZE
from test_framework.key import ECKey, ECPubKey, jacobi_symbol
from test_framework.key import *

# Python Imports
import random
import hashlib
import pdb

# 1.1 Introduction to Schnorr

* Part 1: Elliptic Curve Math Refresher.
* Part 2: Schnorr Signatures.
* Part 3: Deterministic Nonces.

## Part 1: Elliptic Curve Math

Elliptic Curve math involves scalars and Points(X,Y). The following provides an overview of all operations of scalars and points over elliptic curves. In Bitcoin, key pair generation and signing is performed over the SECP256k1 curve.

![test](images/schnorr0.jpg)

### Classes / Methods for Elliptic Curve Math

`int`: All Scalar operations over SECP251k1 can be performed with python integers and the modulo `%` operator:
* Addition: `a + b % SECP256K1`
* Subtraction: `-a = SECP256K1 - a`
* Multiplication: `a * b % % SECP256K1`
* Division (Fermat's little Theorem): `1/b = b**(SECP256K1-2) % SECP256K1`

`ECKey`: The Bitcoin Core library provides a private key class which can also perform certain scalar operations.
* Addition: `add`
* Subtraction: `add` (adding complement of scalar)
* Multiplication: `mul`
* Division: `mul` (multiplying with scalar inverse, see Fermat's little theorem) 

![test](images/schnorr1.jpg)

#### 1.1 Example: Scalar Addition over SECP256K1 order

Addition can be performed with integer modulo math in python or with the private key class `ECKey`. We can set a `ECKey` object to a certain value, or generate a new private key with the `generate` method.

In the example below, addition is performed with both integers and the `ECKey` class, and evaluated for equality.

In [ ]:
# int() operations
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)
ab = (a + b) % SECP256K1_ORDER

# ECkey(operation)
a_key = ECKey()
a_key.set(a.to_bytes(32,'big'), True) 
ab_key = a_key.add(b.to_bytes(32,'big'))

# Ensure equivalency.
print(int().from_bytes(ab_key.get_bytes(),'big') == ab)


#### 1.2 Example: Scalar Multiplication over SECP256K1 order

In the example below, multiplication is performed with both integers and the `ECKey` class, and evaluated for equality.

In [ ]:
# int() operations
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)
ab = (a * b) % SECP256K1_ORDER

# ECkey(operation)
a_key = ECKey()
a_key.set(a.to_bytes(32,'big'), True) 
ab_key = a_key.mul(b.to_bytes(32,'big'))

# Ensure operations are equivalent.
print(int().from_bytes(ab_key.get_bytes(),'big') == ab)


#### 1.3 _Programming Exercise:_ Distributivity of scalar operations

In this exercise we wish to demonstrate the distributivity property of scalar addition and multiplication, whilst getting familiarized with both integer modulo operations and the private key `ECKey` methods.

Consider: `(a - b) * c = a * c - b * c` over SECP256k1
* Demonstrate that the equation holds
* Compute the left side with the Python `int` class.
* Compute the right side with the Bitcoin Core `ECKey` class.


In [ ]:
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)
c = random.randrange(1, SECP256K1_ORDER)

# Left: Compute (a - b) * c


# Right: Compute -b * c
# -b = SECP256K1 - b

# Right: Compute a * c


# Right: Compute a * c - b * c


# Left/Right: Assert equality



#### 1.4 Example: Point Addition over SECP256k1

The public key `ECPubkey` class can be derived from `ECKey` with the `ECKey.get_pubkey` method. 

In the following example, we perform point addition with `ECPubkey.add`.


In [ ]:
a_key = ECKey()
b_key = ECKey()
a_key.generate()
b_key.generate()

A_key = a_key.get_pubkey()
B_key = b_key.get_pubkey()
AB_key = A_key.add(B_key)

print(AB_key.get_bytes().hex())

#### 1.5  _Programming Exercise:_ Distributivity over scalar and ec points.

In this exercise we wish to demonstrate the distributivity property of scalar/point operations, whilst getting familiarized with both integer modulo operations and the public key `ECPubKey` methods.

Consider: `(a - b) * P = a * G - b * G`


In [ ]:
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)

# Left: Compute a - b


# Left: Compute (a - b) * G


# Right: Compute aG


# Right: Compute -bG


# Right: Compute aG - bG


# Left/Right: Assert equality


## Part 2: Schnorr Signatures

The Schnorr signature equation is the following:

* `S = R + H(x(R)|P|msg) * P`

Signing involves generating a secret nonce first.

* Generate secret scalar `k`

Then computing s from:

* `s = k - H(x(R)|P|m)`

The resulting signature is:

* `x(R), s`



![test](images/schnorr2.jpg)



### Constraint on the private nonce k.

The Schnorr signature provides the verifier with the x-coordinate of the Nonce point R and s value. For a given x-coordinate on the SECP251K1 curve, there are two possible curve points:

* `y^2 = x^3 + 7` (Two y-coordinate values for a given x-coordinate)
    * For x, both P(x,y) and P(x,-y) are valid curve points.
    * One of the y-coordinates is even, and the other one is odd.

The nonce k is therefore constrained by requiring it to be a quadratic residue of p (SECP256k1 curve field size). 

* `k` and its complement `SECP26k1_order - k` will have nonce points `R(x, y)` and `R(x, -y)`.
    * Only one will be a quadratic residue of the curve order.
    
Whether a scalar is a quadratic residue of the secp256k1 field size is determined with the jacobi symbol:

* Bitcoin Core method: `jacobi_symbol(k, SECP256K1_FIELD_SIZE) == 1`
* If the jacobisymbol is not 1, simply take the inverse of k: `k = SECP26k1_order - k`


#### 2.1 _Programming Exercise:_ Sign a message with Schnorr

* Sign the message with the provided key pair below.

In [ ]:
msg = hashlib.sha256(b'message').digest()
x = ECKey()
x.generate()
P = x.get_pubkey()

# We have to set "compressed" to false in ECKey.set(data, False) to get the y-coordinate.


# Check that nonce is quadratic residue of field size.
# Method: jacobi_symbol(nonce_int, SECP256K1_FIELD_SIZE)


# Generate s = k + H(R_x|P|msg) * x


# Generate sig = R_x|s


# Verify your signature.
# print(P.verify_schnorr(sig, msg))

## Part 3: Deterministic Nonces for Schnorr signatures

So far we have used a random secret nonce for creating Schnorr signatures. This has the disadvantage that the the user must rely on the robustness of the random generator for each signing rounds. If the nonce generator is biased or even compromised, the private key can be derived for a given signature and known nonce.

Therefore, for the single signer Schnorr signatures, the [Schnorr BIP](https://github.com/sipa/bips/blob/bip-schnorr/bip-schnorr.mediawiki) proposes a deterministic nonce generation scheme.

* `k = sha256(x|msg)`

#### 3.1 _Programming Exercise:_ Signing Schnorr with deterministic nonce.

* Create a Schnorr signature with the deterministic nonce scheme proposed in the Schnorr BIP.
* Compare this signature to the private key class method `ECKey.sign_schnorr(msg)`

In [ ]:
msg = hashlib.sha256(b'message').digest()
x = ECKey()
x.generate()
P = x.get_pubkey()

# We have to set uncompressed to get the y-coordinate.


# Check that nonce is quadratic residue of field size.


# Generate s = k + H(R_x|P|msg) * x


# Generate sig = R_x|s


# Verify/Compare your signatures.

